In [ ]:
# Funky shit
import os
import csv
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/libcudart.so.11.0'
from PIL import Image,ImageOps

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Setup labels as vector outputs rather than digit value
train_labelsVektor = np.zeros((10, len(train_labels)))
for i in range(len(train_labels)):
    for j in range(10):
        if train_labels[i] == j:
            train_labelsVektor[j, i] += 1
train_labelsVektor = np.transpose(train_labelsVektor)

test_labelsVektor = np.zeros((10, len(test_labels)))
for i in range(len(test_labels)):
    for j in range(10):
        if test_labels[i] == j:
            test_labelsVektor[j, i] += 1
test_labelsVektor = np.transpose(test_labelsVektor)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Normalize
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
def generateModel(kernels, kernelDims, poolDims, activationFunctions):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(28, 28, 1), batch_size=32))
    
    for i in range(len(kernels)):
        model.add(tf.keras.layers.Conv2D(kernels[i], kernelDims, activation=activationFunctions))
        model.add(tf.keras.layers.MaxPool2D((poolDims)))
    
    # Dense model
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(56, activation='softplus'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    #model.summary()
    return model

In [ ]:
def compileModel(model):
    model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
    return model

In [ ]:
def trainModel(model, epochs):
    trainAccuracy = np.zeros(epochs)
    testAccuracy = np.zeros(epochs)
    for i in range(epochs):
        print('Epoch iteration: ' + str(i+1))
        with tf.device('/GPU:0'):
            history = model.fit(train_images, train_labelsVektor, epochs=1, batch_size=32, verbose=2)
        trainAccuracy[i] = history.history["accuracy"][0]
        test_loss, test_acc = model.evaluate(test_images,  test_labelsVektor, batch_size=32, verbose=2)
        testAccuracy[i] = test_acc
    return trainAccuracy, testAccuracy

In [ ]:
def writeToCSV(kernels, NNNumber, kernelDims, poolDims, activationFunction, trainAccuracy, testAccuracy):
    with open('./Test_data_CNN/' + activationFunction + '/NN' + str(NNNumber+1) + 'KerDim' + str(kernelDims[0]) + '.csv', 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        writer.writerow(['CNN layers: ' + str(len(kernels))])
        writer.writerow(['Kernels: ' + str(kernels)])
        writer.writerow(['NN: ' + str(NNNumber+1)])
        writer.writerow(['Kernel dim: ' + str(kernelDims)])
        writer.writerow(['Pool dim: ' + str(poolDims)])
        writer.writerow(['Activation function: ' + activationFunction])
        writer.writerow(['Training accuracy', 'Test accuracy'])
        for i in range(len(trainAccuracy)):
            writer.writerow([trainAccuracy[i], testAccuracy[i]])

In [ ]:
# MAX POOL, AVG POOL, STRIDES, 
#kernels = [[28, 28], [28, 56], [14, 28], [28, 112], [28, 56, 56], [28, 56, 112], [56, 112, 112], [28, 56, 112], [14, 28, 56], [112, 224, 224]]
kernels = [[112], [0]]
kernelDims = [2, 2] #[[]
poolDims = [[2, 2], [3, 3]] # bruger kun 2, 2
activationFunctions = ['softplus', 'sigmoid', 'relu']

epochs = 50
#for k in range(len(kernelDims)):
for i in range(len(activationFunctions)):
    for j in range(len(kernels)):
        #print(activationFunctions[i] + ", NN: " + str(j+1) + ", kernelDims: " + str(kernelDims[k]))
        print(activationFunctions[i] + ", NN: " + str(j+1) + ", kernelDims: " + str(kernelDims))
        #model = generateModel(kernels[j], kernelDims[k], poolDims[0], activationFunctions[i])
        model = generateModel(kernels[j], kernelDims, poolDims[0], activationFunctions[i])
        model = compileModel(model)
        trainAccuracy, testAccuracy = trainModel(model, epochs)
        #writeToCSV(kernels[j], j, kernelDims[k], poolDims[0],  activationFunctions[i], trainAccuracy, testAccuracy)
        writeToCSV(kernels[j], j, kernelDims, poolDims[0],  activationFunctions[i], trainAccuracy, testAccuracy)

In [ ]:
range(len(kernelDims))